In [1]:
from gutenberg.cleanup import strip_headers
import requests
# http://nbviewer.jupyter.org/github/rare-technologies/gensim/blob/develop/docs/notebooks/atmodel_tutorial.ipynb

def load_etext(num):
  url ='http://www.gutenberg.org/files/{0}/{0}-0.txt'.format(num)
  data= requests.get(url)
  return data.text

In [3]:
texts =[]

text_nums=[1342, 158, 121, 105, 141,
           76,74, 86, 3176, 8525, 245,
           11,12, 29042, 620,35535,
           2701,21816, 34970, 12384,2694,
           98,19337, 786, 1023, 1400, 730, 
           1795, 2265, 2251, 2253, 1794, 1514, 1793, 1513, 1801,
           2814, 4217, 4300,
           5200,
           2554,
           84,
           1399,
           140,
           345,
           174
         ]
text_names =['Pride and Prejudice', 'Emma', 'Northanger Abbey', "Persuasion", "Mansfield Park",
             'Adventures of Huckleberry Finn', 'The Adventures of Tom Sawyer', "A Connecticut Yankee in King Arthur's Court", "The Innocents Abroad", "Eve's Diary", "Life on the Mississippi", 
             "Alice in Wonderland","Through the Looking-Glass", "A Tangled Tale", "Sylvie and Bruno", "Feeding the Mind",
            "Moby Dick", "The Confidence-Man: His Masquerade", "Pierre; or The Ambiguities", "Battle-Pieces and Aspects of the War","I and My Chimney",
             "A Tale of Two Cities","A Christmas Carol","Hard Times", "Bleak House", "Great Expectations", "Oliver Twist",
             "MacBeth", "Hamlet", "Henry IV, Part 1", "Henry V", "King Lear", "A Midsummer Night's Dream", "Othello", "Romeo and Juliet", "The Tempest",
             "Dubliners", "A Portrait of the Artist as a Young Man", "Ulysses",
             "Metamorphosis",
             "Crime and Punishment",
             "Frankenstein; Or, The Modern Prometheus",
             "Anna Karenina",
             "The Jungle",
             "Dracula",
             "The Picture of Dorian Gray"
            ]
for text_num in text_nums:
    text = strip_headers(load_etext(text_num)).strip()
    texts.append(text)

# Preprocess Text

- Lowercase
- Strip Tags
- Strip Punctuation
- Strip Multiple Whitespaces
- Strip Numeric
- Remove Stopwords
- Strip Shortwords
- Stem Text


In [4]:
from gensim.parsing.preprocessing import preprocess_string,preprocess_documents
from gensim import corpora,models, similarities


D:\Drace\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
processed_docs = preprocess_documents(texts)

In [6]:
dictionary = corpora.Dictionary(processed_docs)
dictionary.save('gutenberg.dict')

In [7]:
print(dictionary)

Dictionary(44269 unique tokens: [u'fawn', u'\u2014\u2018the', u'piffpaff', u'\u2014peep', u'mustachio']...)


In [8]:
corpus = [dictionary.doc2bow(text) for text in processed_docs]
corpora.MmCorpus.serialize('gutenberg.mm', corpus) 

# After some model tweaking and different query

In [195]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=75, iterations= 1000, alpha= 'asymmetric')
lda.save('ldagutenberg.model')
indexLda = similarities.MatrixSimilarity(lda[corpus])
indexLda.save('ldagutenberg.index')

In [196]:
def queryLDA(doc):
    vec_bow = dictionary.doc2bow(preprocess_string(doc))
    vec_lda = lda[vec_bow] 
    return vec_lda

In [197]:
query_texts = ["""
ip’s story—the story of the novel—traces his development through the events of his early life; his narration, however, written years after the end of the story, is a product of his character as it exists after the events of the story. Pip’s narration thus reveals the psychological endpoint of his development in the novel. Pip’s behavior as a character often reveals only part of the story—he treats Joe coldly, for instance—while his manner as a narrator completes that story: his guilt for his poor behavior toward his loved ones endures, even as he writes about his early life years later. Of course, Dickens manipulates Pip’s narration in order to evoke its subjects effectively: Pip’s childhood is narrated in a much more childlike voice than his adult years, even though the narrator Pip presumably writes both parts of the story at a single later date. Dickens also uses Pip’s narration to reinforce particular aspects of his character that emerge in the course of the novel: we know from his actions that Pip is somewhat self-centered but sympathetic at heart to others; Pip’s later narration of his relationships with others tends to reflect those qualities. When Magwitch reveals that he is Pip’s benefactor, for instance, Pip is disgusted by the convict and describes him solely in negative terms; as his affection for Magwitch grows, the descriptive terms he chooses to apply to the convict become much more positive.
"""""]
query_LDA = map(queryLDA, query_texts)

In [198]:
query_LDA[0]

[(5, 0.9413487907073399), (21, 0.050497952099912397)]

In [199]:
simsLDA = indexLda[query_LDA[0]]

In [200]:
sorted(zip(text_names,simsLDA),key=lambda x: x[1],reverse=True)

[('Sylvie and Bruno', 0.51769108),
 ("A Connecticut Yankee in King Arthur's Court", 0.44035211),
 ('Great Expectations', 0.36185449),
 ('A Tale of Two Cities', 0.31765339),
 ('The Innocents Abroad', 0.31264672),
 ('Life on the Mississippi', 0.29334447),
 ('Moby Dick', 0.1059795),
 ('Anna Karenina', 0.039721411),
 ('Northanger Abbey', 0.022235036),
 ('Pride and Prejudice', 0.0),
 ('Emma', 0.0),
 ('Persuasion', 0.0),
 ('Mansfield Park', 0.0),
 ('Adventures of Huckleberry Finn', 0.0),
 ('The Adventures of Tom Sawyer', 0.0),
 ("Eve's Diary", 0.0),
 ('Alice in Wonderland', 0.0),
 ('Through the Looking-Glass', 0.0),
 ('A Tangled Tale', 0.0),
 ('Feeding the Mind', 0.0),
 ('The Confidence-Man: His Masquerade', 0.0),
 ('Pierre; or The Ambiguities', 0.0),
 ('Battle-Pieces and Aspects of the War', 0.0),
 ('I and My Chimney', 0.0),
 ('A Christmas Carol', 0.0),
 ('Hard Times', 0.0),
 ('Bleak House', 0.0),
 ('Oliver Twist', 0.0),
 ('MacBeth', 0.0),
 ('Hamlet', 0.0),
 ('Henry IV, Part 1', 0.0),
 ('He

In [201]:
query_texts1 = ['''The Pequod leaves Nantucket on a cold Christmas Day with a crew made up of men from many different countries and races. Soon the ship is in warmer waters, and Ahab makes his first appearance on deck, balancing gingerly on his false leg, which is made from a sperm whale’s jaw. He announces his desire to pursue and kill Moby Dick, the legendary great white whale who took his leg, because he sees this whale as the embodiment of evil. Ahab nails a gold doubloon to the mast and declares that it will be the prize for the first man to sight the whale. As the Pequod sails toward the southern tip of Africa, whales are sighted and unsuccessfully hunted. During the hunt, a group of men, none of whom anyone on the ship’s crew has seen before on the voyage, emerges from the hold. The men’s leader is an exotic-looking man named Fedallah. These men constitute Ahab’s private harpoon crew, smuggled aboard in defiance of Bildad and Peleg. Ahab hopes that their skills and Fedallah’s prophetic abilities will help him in his hunt for Moby Dick.

The Pequod rounds Africa and enters the Indian Ocean. A few whales are successfully caught and processed for their oil. From time to time, the ship encounters other whaling vessels. Ahab always demands information about Moby Dick from their captains. One of the ships, the Jeroboam, carries Gabriel, a crazed prophet who predicts doom for anyone who threatens Moby Dick. His predictions seem to carry some weight, as those aboard his ship who have hunted the whale have met disaster. While trying to drain the oil from the head of a captured sperm whale, Tashtego, one of the Pequod’s harpooners, falls into the whale’s voluminous head, which then rips free of the ship and begins to sink. Queequeg saves Tashtego by diving into the ocean and cutting into the slowly sinking head.''']
query_LDA1 = map(queryLDA, query_texts1)
simsLDA1 = indexLda[query_LDA1[0]]
sorted(zip(text_names,simsLDA1),key=lambda x: x[1],reverse=True)
#should give Moby Dick

[('Moby Dick', 0.85463953),
 ('Ulysses', 0.050821528),
 ('Sylvie and Bruno', 0.033483319),
 ('Hard Times', 0.020253118),
 ('A Tale of Two Cities', 0.01743748),
 ('Mansfield Park', 0.0064144032),
 ('The Innocents Abroad', 0.0007874929),
 ('Pride and Prejudice', 0.0),
 ('Emma', 0.0),
 ('Northanger Abbey', 0.0),
 ('Persuasion', 0.0),
 ('Adventures of Huckleberry Finn', 0.0),
 ('The Adventures of Tom Sawyer', 0.0),
 ("A Connecticut Yankee in King Arthur's Court", 0.0),
 ("Eve's Diary", 0.0),
 ('Life on the Mississippi', 0.0),
 ('Alice in Wonderland', 0.0),
 ('Through the Looking-Glass', 0.0),
 ('A Tangled Tale', 0.0),
 ('Feeding the Mind', 0.0),
 ('The Confidence-Man: His Masquerade', 0.0),
 ('Pierre; or The Ambiguities', 0.0),
 ('Battle-Pieces and Aspects of the War', 0.0),
 ('I and My Chimney', 0.0),
 ('A Christmas Carol', 0.0),
 ('Bleak House', 0.0),
 ('Great Expectations', 0.0),
 ('Oliver Twist', 0.0),
 ('MacBeth', 0.0),
 ('Hamlet', 0.0),
 ('Henry IV, Part 1', 0.0),
 ('Henry V', 0.0),


In [202]:
query_texts2 = ['''The intricacies and tedium of high society, particularly partner selection.
The conflicts of marriage for love and marriage for property.
Life lived as if in a Gothic novel, filled with danger and intrigue, and the obsession with all things Gothic.
The dangers of believing life is the same as fiction.
The maturation of the young into skeptical adulthood, the loss of imagination, innocence and good faith.
Things are not what they seem at first.
Social criticism (comedy of manners).
Parody of the Gothic novels' "Gothic and anti-Gothic" attitudes.''']
query_LDA2 = map(queryLDA, query_texts2)
simsLDA2 = indexLda[query_LDA2[0]]
sorted(zip(text_names,simsLDA2),key=lambda x: x[1],reverse=True)

#should give Northanger Abbey.

#BIG SUCCESS.  This was only a theme outline from wikipedia with NO mention of words from the book and it was able to correctly identify the text

[('Sylvie and Bruno', 0.80725151),
 ('Ulysses', 0.5910477),
 ('A Tale of Two Cities', 0.43241709),
 ("A Connecticut Yankee in King Arthur's Court", 0.35542217),
 ('Great Expectations', 0.29206425),
 ('The Innocents Abroad', 0.26150551),
 ('Life on the Mississippi', 0.23676762),
 ('Hard Times', 0.23554109),
 ('Moby Dick', 0.085539415),
 ('Mansfield Park', 0.07459867),
 ('Northanger Abbey', 0.017946603),
 ('Pride and Prejudice', 0.0),
 ('Emma', 0.0),
 ('Persuasion', 0.0),
 ('Adventures of Huckleberry Finn', 0.0),
 ('The Adventures of Tom Sawyer', 0.0),
 ("Eve's Diary", 0.0),
 ('Alice in Wonderland', 0.0),
 ('Through the Looking-Glass', 0.0),
 ('A Tangled Tale', 0.0),
 ('Feeding the Mind', 0.0),
 ('The Confidence-Man: His Masquerade', 0.0),
 ('Pierre; or The Ambiguities', 0.0),
 ('Battle-Pieces and Aspects of the War', 0.0),
 ('I and My Chimney', 0.0),
 ('A Christmas Carol', 0.0),
 ('Bleak House', 0.0),
 ('Oliver Twist', 0.0),
 ('MacBeth', 0.0),
 ('Hamlet', 0.0),
 ('Henry IV, Part 1', 0.0

# Query

notes:
150 topics, 2000 iterations, model cannot capture the general themes.  Too many topics drown out the specifics.
140 topics, 2000 iterations, model lost accuracy in all fields
130 topics, 2000 iterations, model became very inaccurate, going to jump iterations
130 topics, 2500 iterations, model performed roughly as well as 150 topics, 2000 iterations
100 topics, 2500 iterations, model performed worse than previous but significantly more confident in prediction of term paper Moby Dick
75 topics, 2500 iterations, model performs similarly to 130, 2500
50, 2500 iterations, model worse performance
50, 3000 iterations, model predicts correctly with confidence in q1,2 and incorrectly with confidence 3
50, 4K iterations, model worse performance
150, 4K iterations, slight improvement scores on q1,2, incorrect on 3
200, 4K iterations, worse in all scores
200, 1K iterations, incorrect confident on q1, correct confident on 2,3
75, 1K iterations, no improvement.

will need to save log tomorrow